In [5]:
!pip install openpyxl

In [48]:
import pandas as pd
from datetime import datetime
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [12]:
df = df = pd.read_excel('Online_Retail.xlsx')

In [21]:
# Fetch the first 5 rows from the dataset
df.tail(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


## Let's clean the data

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    532618 non-null  float64       
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
 8   Date         541909 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(3), int64(1), object(3)
memory usage: 37.2+ MB


In [45]:
df.describe()

,InvoiceNo,Quantity,InvoiceDate,UnitPrice,CustomerID,Date
count,532618.000000,541909.000000,541909,541909.000000,406829.000000,541909
mean,559965.752027,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570,2011-07-04 13:34:57.156386048
min,536365.000000,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000,2010-12-01 08:26:00
25%,547906.000000,1.000000,2011-03-28 11:34:00,1.250000,13953.000000,2011-03-28 11:34:00
50%,560688.000000,3.000000,2011-07-19 17:17:00,2.080000,15152.000000,2011-07-19 17:17:00
75%,571841.000000,10.000000,2011-10-19 11:27:00,4.130000,16791.000000,2011-10-19 11:27:00
max,581587.000000,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000,2011-12-09 12:50:00
std,13428.417281,218.081158,NaN,96.759853,1713.600303,NaN


In [29]:
# Check for duplicates

In [34]:
df.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
541904    False
541905    False
541906    False
541907    False
541908    False
Length: 541909, dtype: bool

In [35]:
#Check NaN Values

In [36]:
df.isnull().values.any()

True

In [42]:
# Check for missing values

missing_values = df.isnull().sum()
print(missing_values)

InvoiceNo        9291
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
Date                0
dtype: int64


In [47]:
df.dtypes

InvoiceNo             float64
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
Date           datetime64[ns]
dtype: object